# Objectives
The purpose of this section is to implement a dictionary attack over a WPA2-Personal wifi.To answer most of the following questions, we used the Python cryptography library.

# Preliminaries
File tradio.pcapng is a capture test that contains a WPA2-Personal handshake and some ping packets after handshake for the following wifi session:

*   SSID: Wifi Test
*   Password: Wifi Test Password

You can identify the 4 handshake messages by applying filter eapol in wireshark. Also, you can
decrypt traffic (see: https://wiki.wireshark.org/HowToDecrypt802.11).

# Question 1
For WPA2 personal, PMK is PSK. Prove that PMK is: a22c...b603

The first thing we have to do is to derive the PMK or also called PSK from:


*   SSID
*   Password




In [3]:
from cryptography.hazmat.primitives.kdf.pbkdf2  import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes

ssid = "Wifi_Test"  # SSID (Wi-Fi network name)
password = "Wifi_Test_Password"     # Wi-Fi password

# Generate salt based on the SSID
salt = ssid.encode()

# Define parameters for PBKDF2 key derivation function
key_length = 32    # Length of the derived key (PMK) in bytes
iterations = 4096  # Number of iterations

# Derive PMK using PBKDF2HMAC key derivation function
kdf = PBKDF2HMAC(
        algorithm=hashes.SHA1(),
        length=key_length,
        salt=salt,
        iterations=iterations,
)
pmk = kdf.derive(password.encode())

# Convert PMK bytes to hexadecimal representation
pmk_hex = pmk.hex()
print("Derived PMK:", pmk_hex)

Derived PMK: a22c4740d1843aca77252e5a6923af13773be449bb443edca5c6296ca151b603


# Question 2.
 Parse the capture file 'tradio.pcapng' with tshark and dump the required handshake packets in hexadecimal. From the first handshake message, prove that nonce is b6ea...d7d3b. From the second handshake message, prove that nonce is e6d8...b61c.


Instead of using tshark, we decided to use pyshark to parse the captured file and also dump the required handshake packets.


In [10]:
import pyshark
import nest_asyncio

nest_asyncio.apply()

handshake_messages = pyshark.FileCapture(input_file="tradio.pcapng", display_filter='eapol')

# Set use_json and include_raw to True
capture = pyshark.FileCapture(input_file="tradio.pcapng", display_filter='eapol', use_json=True, include_raw=True)

print("1rst handshake:" + handshake_messages[0].eapol.wlan_rsna_keydes_nonce.replace(":",""))
print("2nd handshake:" + handshake_messages[1].eapol.wlan_rsna_keydes_nonce.replace(":",""))


1rst handshake:b6ea13c99b3a6aeeb973b622d5671d3f4a143effe67660c0bc01b774604d7d3b
2nd handshake:e6d818b941847156d83eab024dc81f8ce372ab4a625bf0f80c383c536578b61c


# Question 3.
 Prove that PTK is 0ffc...8aad.

First of all we need to extract de MAC address and nonce from the capture.

In [5]:


MAC_add1 = handshake_messages[0].wlan.ra.replace(":","")
MAC_add2 = handshake_messages[0].wlan.ta.replace(":","")
nonce_1 = handshake_messages[0].eapol.wlan_rsna_keydes_nonce.replace(":","")
nonce_2 = handshake_messages[1].eapol.wlan_rsna_keydes_nonce.replace(":","")

print("MAC_add1: " + MAC_add1, "\nonce_1: " + nonce_1, "\nMAC_add2: " + MAC_add2, "\nnonce_2: " + nonce_2)


MAC_add1: 2269a9e50b35 
once_1: b6ea13c99b3a6aeeb973b622d5671d3f4a143effe67660c0bc01b774604d7d3b 
MAC_add2: 84aa9cfd0820 
nonce_2: e6d818b941847156d83eab024dc81f8ce372ab4a625bf0f80c383c536578b61c


Once we have those, we can calculate the ptk, using the provided functions from the diapos (100).

In [6]:
from cryptography.hazmat.primitives import hmac

def HMAC(key, algo, input):
    h = hmac.HMAC(key, algo)
    h.update(input)
    return h.finalize()

def SHA1_HMAC(key, input, length):
    r = bytes()
    for i in range(1 + length // 20):
        r+= HMAC(key, hashes.SHA1(), input + chr(i).encode())
    return r[:length]

# ordered the fields (less first)
mac_1, mac_2 = min(MAC_add1, MAC_add2) , max(MAC_add1, MAC_add2)
nonce_1, nonce_2 = min(nonce_1, nonce_2), max(nonce_1, nonce_2)
print(type(mac_1))
data = 'Pairwise key expansion'.encode() + chr(0x00).encode() + bytes.fromhex(mac_1) + bytes.fromhex(mac_2) + bytes.fromhex(nonce_1)+ bytes.fromhex(nonce_2)

PTK = SHA1_HMAC(pmk, data, 48)

print(PTK.hex())

<class 'str'>
0ffc1910a2ff9e32a260c3e10427ff980749105d9cfdf27cf232798126a306ef5d75e703a06de9d51b492dfdb9138aad


# Question 4.
Prove the authenticity of handshake messages 2, 3 and 4.

First of all we have to extract the KCK, KEK and TK keys from the PTK.

In [7]:
KCK = PTK[:16]
KEK = PTK[16:32]
TK = PTK[32:48]

Once we have those keys, we can compute the MIC, which then we can verify if it's equal to the MIC of the packet.

In [134]:
def compute_mic(KCK, mess):
    data = mess
    for _ in range(16):
        data+=chr(0x00).encode()
    data += mess
    return HMAC(KCK, hashes.SHA1(), data)[:16]

To verify if it's the same, we have to search in the packets for the MIC and compare it with the results.

In [147]:
print(handshake_messages[1].eapol._all_fields)

handshake_message_1_eapol = handshake_messages[1].eapol._all_fields
print(handshake_message_1_eapol)
handshake_message_2_eapol = handshake_messages[2].eapol
handshake_message_3_eapol = handshake_messages[3].eapol

handshake_message_1_mic = compute_mic(KCK, handshake_message_1_eapol)
handshake_message_2_mic = compute_mic(KCK, handshake_message_2_eapol)
handshake_message_3_mic = compute_mic(KCK, handshake_message_3_eapol)

print('Computed: ' +  handshake_message_1_mic + 'Expected: ' + handshake_messages[1].eapol.wlan_rsna_keydes_mic.replace(":",""))
print('Computed: ' +  handshake_message_2_mic + 'Expected: ' + handshake_messages[2].eapol.wlan_rsna_keydes_mic.replace(":",""))
print('Computed: ' +  handshake_message_3_mic + 'Expected: ' + handshake_messages[3].eapol.wlan_rsna_keydes_mic.replace(":",""))

{'eapol.version': '1', 'eapol.type': '3', 'eapol.len': '117', 'eapol.keydes.type': '2', 'wlan_rsna_eapol.keydes.msgnr': '2', 'wlan_rsna_eapol.keydes.key_info': '0x010a', 'wlan_rsna_eapol.keydes.key_info.keydes_version': '2', 'wlan_rsna_eapol.keydes.key_info.key_type': '1', 'wlan_rsna_eapol.keydes.key_info.key_index': '0', 'wlan_rsna_eapol.keydes.key_info.install': '0', 'wlan_rsna_eapol.keydes.key_info.key_ack': '0', 'wlan_rsna_eapol.keydes.key_info.key_mic': '1', 'wlan_rsna_eapol.keydes.key_info.secure': '0', 'wlan_rsna_eapol.keydes.key_info.error': '0', 'wlan_rsna_eapol.keydes.key_info.request': '0', 'wlan_rsna_eapol.keydes.key_info.encrypted_key_data': '0', 'wlan_rsna_eapol.keydes.key_info.smk_message': '0', 'eapol.keydes.key_len': '0', 'eapol.keydes.replay_counter': '0', 'wlan_rsna_eapol.keydes.nonce': 'e6:d8:18:b9:41:84:71:56:d8:3e:ab:02:4d:c8:1f:8c:e3:72:ab:4a:62:5b:f0:f8:0c:38:3c:53:65:78:b6:1c', 'eapol.keydes.key_iv': '00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00', 'wlan_rsna

TypeError: unsupported operand type(s) for +=: 'dict' and 'bytes'

# 1.3 Decrypt unicast data
Now is time to decrypt a data packet. Note that it is a QoS Data Frame. You can use packet
number 517 which is a ping request. Encryption is done using AES128-CCMP.

# Question 5.
Prove that nonce is: 002269a9e50b3500000000000b
First of all we have to find and parse the 517 packet.

In [34]:
packets = pyshark.FileCapture(input_file="tradio.pcapng")
packet_count = 0
for packet in packets:
    packet_count += 1
    if(packet_count == 517):
        pckg_517 = packet

Once we have the packet, to prove the nonce we have to compute it, according to the diapo (101) we have to consider that the frame has a QoS field, so we compute it like this.
nonce = QoSControl[0:1] + addr_2 + CCMPpar[0:2] + CCMpar[4:8]


In [82]:
QoSControl = pckg_517.wlan.qos
CCMPpar = pckg_517.wlan.ccmp_extiv
nonce = QoSControl[0:1] + pckg_517.wlan.ta + CCMPpar[0:2] + CCMPpar[4:8]
print(len(nonce.replace(":","")),nonce.replace(":",""))

19 02269a9e50b350x0000


# Question 6.
Prove that AAD is: 884184aa9cfd08202269a9e50b3584aa9cfd081f00000000
According to the diapo (101) we have to consider that the frame has a QoS field, so we compute it like this.
AAD = FrameControl (2 bytes) || Address 1 (6 bytes) || Address 2 || Address 3 ||0x0000 || QoSControl[0:2]


In [87]:
fc = pckg_517.wlan.fc.replace(":","").encode()
add_1 = pckg_517.wlan.ra.replace(":","").encode()
add_2 = pckg_517.wlan.ta.replace(":","").encode()
add_3 = pckg_517.wlan.bssid.replace(":","").encode()

AAD = fc + add_1 + add_2 + add_3 + chr(0x00).encode() + QoSControl[0:2].encode()
print(AAD)

b'0x884184aa9cfd08202269a9e50b3584aa9cfd0820\x000x'


# Question 7.
Prove that decrypted plaintext is: aaaa...3637
According to the diapo (101) we have to consider that it's a unicast packet, so we compute it like this.
aesccm = AESCCM(key) and aesccm.decrypt(nonce, ct, aad), where the key is TK.


In [103]:
from cryptography.hazmat.primitives.ciphers.aead import AESCCM

data_to_decrypt = pckg_517.data.data_data.replace(":","").encode()
unicast_nonce = nonce.replace(":","").encode() #We take the previous derived nonce.

aesccm = AESCCM(TK, tag_length=8)

decrypted_data= aesccm.decrypt(unicast_nonce[:13], data_to_decrypt, AAD)

print(decrypted_data)

16


InvalidTag: 

# 1.4 Decrypt multicast/broadcast data
In the same capture, there is also multicast/broadcast traffic, such as packet 527, that can not
be decrypted with the same key. Note that it is not a QoS Data Frame. First we have to parse the packet 527, like we did before with the 517.

In [40]:
packets = pyshark.FileCapture(input_file="tradio.pcapng")
packet_count = 0
for packet in packets:
    packet_count += 1
    if(packet_count == 527):
        pckg_527 = packet

## Question 8.
Obtain GTK and prove that is: 3014...dd00. 

Once we have the packet, in order to obtain the GTK, we need to unwrap it from the 3rd handshake message, like this.
GTK=AES UNWRAP(key=KEK, in=data)[30:46]

In [68]:
from cryptography.hazmat.primitives.keywrap import aes_key_unwrap
wrapped_key = handshake_messages[2].eapol.wlan_rsna_keydes_data.replace(":","")

GTK = aes_key_unwrap(KEK,bytes.fromhex(wrapped_key))

print(GTK.hex())


30140100000fac040100000fac040100000fac020c00dd16000fac010100f3b1ceb5e1d4d5e8ade18749e4cc4ec7dd00


## Question 9.
Prove that nonce is: 0084aa9cfd08200000000008f0


In [52]:
add_2 = pckg_527.wlan.sa.replace(":","")
CCMPpar = pckg_527.wlan.ccmp_extiv
    
multicast_nonce = chr(0x00).encode() + add_2.encode() + CCMPpar[0:2].encode() + CCMPpar[4:8].encode()

print(multicast_nonce)

b'\x0084aa9cfd081f0x0000'


## Question 10.
Compute the ADD of the multicast package. We know that is not a QoS data frame so, we compute it like this: AAD = FrameControl || Address 1 || Address 2 || Address 3 || 0x0000

In [73]:
fc = pckg_527.wlan.fc.replace(":","").encode()
add_1 = pckg_527.wlan.ra.replace(":","").encode()
add_2 = pckg_527.wlan.sa.replace(":","").encode()
add_3 = pckg_527.wlan.bssid.replace(":","").encode()

Multicast_ADD = fc + add_1 + add_2 + add_3 + chr(0x00).encode()
print(len(Multicast_ADD), Multicast_ADD)
print(len('0842ffffffffffff84aa9cfd082084aa9cfd081f0000'), '0842ffffffffffff84aa9cfd082084aa9cfd081f0000')

43 b'0x0842ffffffffffff84aa9cfd081f84aa9cfd0820\x00'
44 0842ffffffffffff84aa9cfd082084aa9cfd081f0000


## Question 11.
Decrypt packet 527. To decryp this packet, we follow the previous packet decryption but this time instead of using the TK key, we use the GTK which is the key that protects multicast and broadcast traffic.

In [67]:
data_to_decrypt = pckg_527.data.data_data.replace(":","").encode()


aesccm = AESCCM(GTK, tag_length=8)

multicast_decrypted_data= aesccm.decrypt(multicast_nonce[:13], data_to_decrypt, AAD)

print(multicast_decrypted_data)

ValueError: AESCCM key must be 128, 192, or 256 bits.

# 1.5 Dictionary attack
File tradio2.pcapng contains traffic of a wifi network with SSID: Wifi Test but password is unknown. We only now that password consists of the SSID concatenated to a number, having 10 possible passwords.

## Question 12. 
Write a dictionary attack based on checking the integrity of message 2 to derive the correct password. First of all we have to parse the new capture.

In [105]:
capture = pyshark.FileCapture(input_file="tradio2.pcapng")

Once we have parsed the capture, according to the statement, the password is the SSID (Wifi_Test) concatenated to a number and knowing that there are 10 possible passwords, we can deduce that this number will go from 0 to 9 (10 possibilitiies). We will apply this ten possibilities, and try to find the password on the handshake messages, trying to compute the MIC with the expected one from the 2n handhshake package.

In [129]:
SSID = 'Wifi_Test'

for i in range(0,10):
    # password to test
    password = f'{SSID}{i}'

    # Generate salt based on the SSID
    salt = SSID.encode()

    # Define parameters for PBKDF2 key derivation function
    key_length = 32    # Length of the derived key (PMK) in bytes
    iterations = 4096  # Number of iterations

    # Derive PMK using PBKDF2HMAC key derivation function
    kdf = PBKDF2HMAC(
            algorithm=hashes.SHA1(),
            length=key_length,
            salt=salt,
            iterations=iterations,
    )
    pmk = kdf.derive(password.encode())

    # Convert PMK bytes to hexadecimal representation
    pmk_hex = pmk.hex()

    # We need to catch the handshake samples
    handshake_messages = pyshark.FileCapture(input_file="tradio.pcapng", display_filter='eapol')

    MAC_add1 = handshake_messages[0].wlan.ra.replace(":","")
    MAC_add2 = handshake_messages[0].wlan.ta.replace(":","")
    nonce_1 = handshake_messages[0].eapol.wlan_rsna_keydes_nonce.replace(":","")
    nonce_2 = handshake_messages[1].eapol.wlan_rsna_keydes_nonce.replace(":","") 

    # ordered the fields (less first)
    mac_1, mac_2 = min(MAC_add1, MAC_add2) , max(MAC_add1, MAC_add2)
    nonce_1, nonce_2 = min(nonce_1, nonce_2), max(nonce_1, nonce_2)         
    
    # generate the data blocks as we did in question 3
    data = 'Pairwise key expansion'.encode() + chr(0x00).encode() + bytes.fromhex(mac_1) + bytes.fromhex(mac_2) + bytes.fromhex(nonce_1)+ bytes.fromhex(nonce_2)

    # ptk and subkeys
    PTK = SHA1_HMAC(pmk, data, 48)
    KCK = PTK[:16]
    KEK = PTK[16:32]
    TK = PTK[32:48]

    # catch the mic of the second package to compare it with the computed
    expected_mic = handshake_messages[1].eapol.wlan_rsna_keydes_mic.replace(":","")
    computed_mic = compute_mic(KCK,)

    if(expected_mic == computed_mic):
        print(password)
    else: 
        print('Password not found')




27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
27c403fe1ebc7060c97748972dc3582c
